In [ ]:
#Read train and depth.csv and put them in one dataframe df_Train
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from PIL import Image
import matplotlib.image as mpimg
import os
import random as rn
import tensorflow as tf
from keras import backend as K

#set random seed to get reproducible results
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1234)
rn.seed(1234)
num_train_images= 4000

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print(os.listdir("../input"))

df_train = pd.read_csv("../input/train.csv")
#print(df_train.head())
print("\n Train files shape is ", df_train.shape)

df_depths = pd.read_csv("../input/depths.csv")

df_depths['z'] = df_depths['z'] / np.max(df_depths['z'])
#print(df_depths.head())
print("\nDepths files shape is ", df_depths.shape)

df_train = df_train.join(df_depths, lsuffix='idl', rsuffix='idr', how ='inner')
df_train.pop('ididr')
df_train.columns=['id', 'rle_mask','depth']
#print(df_train.head())
                        


In [ ]:
#2. Read train & mask images and add them up df_train 
df_train['images'] = [np.array(Image.open("../input/train/images/{}.png".format(idx))) for 
                               idx in df_train['id']]
print("Sample Image Shape is ", df_train['images'][0].shape)

df_train['images'] = pd.Series(map(lambda x: np.delete(x,np.s_[1:],2), df_train['images']))
print("After optimization,  Image Shape is ", df_train['images'][0].shape)
print("No. of train images are ", len(df_train))

df_train['images']=df_train['images']/255
print("After normalization,  pixel value is ", df_train['images'][0][1,0])

df_train['masks'] = [np.array(Image.open("../input/train/masks/{}.png".format(idx))) for 
                               idx in df_train['id']]
print("\nSample Mask Shape is ", df_train['masks'][0].shape)

#df_train['masks']=df_train['masks']/255
print("No. of mask images are ", len(df_train))
print("Before normalization,  pixel value is ", df_train['masks'][15][10,0])

print("train df columns are ", df_train.columns)

In [ ]:
train_x = np.array(df_train['images'])
train_x = np.concatenate(train_x,axis=0)
train_x = np.reshape(train_x,(4000,101,101,1))
print("Train Shape = ",train_x.shape)
print("Sample Pixel VAlue", train_x[0,1,0])
#print(df_train['masks'][5][0])

train_y = np.array(df_train['masks'])
train_y = np.concatenate(train_y,axis=0)
train_y = np.reshape(train_y,(4000,101,101,1))
train_y = train_y / train_y.max()
print("Mask Shape = ",train_y.shape)
train_y = np.round(train_y)
print("Sample Pixel VAlue", train_y[15,10,0])

In [ ]:
#train_x = np.append(train_x, [np.fliplr(x) for x in train_x], axis=0)
#train_y = np.append(train_y, [np.fliplr(x) for x in train_y], axis=0)
print(train_x.shape)
#print(train_x_hflip.shape)
print(train_y.shape)



In [ ]:
import os
import scipy.signal as sg
import tensorflow as tf
from keras.models import Model
from keras import layers, regularizers
from keras import backend as K
import numpy as np
from keras import layers
Height = 101
Width = 101



In [ ]:
import random as rn
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1234)
rn.seed(1234)
num_train_images= 4000
#print(depth_np.shape)
#depth_np = np.zeros(shape = (num_train_images,Height,Width,1))
#for i in range(num_train_images):
#    depth_np[i] = np.full(shape=(Height,Width,1), fill_value=df_train['depth'].loc[i]) for 

df_train['depth_image'] = pd.Series(map(lambda x:np.full(shape=(50,50,1), 
                                                         fill_value=x), df_train['depth']))

depth_np = np.array(df_train['depth_image'])
depth_np = np.concatenate(depth_np,axis=0)
depth_np = np.reshape(depth_np,(num_train_images,50,50,1))
depth_np_new = np.array(df_train['depth'])
print("depth", depth_np_new[0:5])
print(depth_np_new.shape)
depth_np_new = depth_np_new.reshape((num_train_images,1,1,1))
print("depth shape ", depth_np_new.shape)
#print(a.shape)
depth_np= depth_np/depth_np.max()
#print(a[2][2])
print(df_train.columns)
#depth_np = np.concatenate((depth_np, depth_np))
print(depth_np.shape)


In [ ]:
Activation1 = 'elu'
Activation2 = 'tanh'
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = layers.BatchNormalization()(x)
        x = layers.Activation(Activation1)(x)
    return x

def residual_block(blockInput, num_filters=16, activate = True):
    #x = BatchActivate(blockInput)
    x = convolution_block(blockInput, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = layers.Add()([x, blockInput])
    x = layers.BatchNormalization()(x)
    if activate:
        x = layers.Activation(Activation2)(x)
    return x

In [ ]:
reg  = regularizers.l2(0.01)
img_input = layers.Input(shape=(Height, Width,1), name = 'img_input')
depth_input = layers.Input(shape=(50,50,1), name='depth_input')
depth_input_new = layers.Input(shape=(1,1,1), name='depth_input_new')

print(img_input)
print(depth_input)

# First convolution extracts 16 filters that are 3x3 for Image Input and Depth Input
#101-> 100
print("101->100")
x = layers.Conv2D(filters = 16, kernel_size= (2,2), padding='valid', activation='elu')(img_input)
print(x)

x_depth = layers.Conv2D(filters = 16, kernel_size= (2,2), padding='valid', activation='elu')(depth_input)
print(x_depth)

# Convolution is followed by max-pooling layer with a 2x2 window for Image Input and Avg Pooling for Depth Input
print("100 -> 50") #100-> 50
x_16_pool = layers.MaxPooling2D(pool_size=(2,2))(x)
x_16_pool = layers.BatchNormalization()(x_16_pool)
#x_16_pool = layers.Dropout(0.25)(x_16_pool) #Removed
print(x)

x_16_pool_depth = layers.AveragePooling2D(pool_size=(2,2))(x_depth)
x_16_pool_depth = layers.BatchNormalization()(x_16_pool_depth)
x_16_pool_depth = layers.Dropout(0.2)(x_16_pool_depth)
print(x_16_pool_depth)

# Second convolution with 2 same layers and resnet extracts 32 filters that are 3x3
print("50-> 48") #50 > 48
x = layers.Conv2D(32, 3, padding='valid',activation='elu')(x_16_pool)
#x = residual_block(x,num_filters = 32) 
x2 = layers.Conv2D(32, 3, padding='same',activation='tanh', kernel_regularizer=reg)(x) 
x = layers.add([x, x2])
x = layers.BatchNormalization()(x)
print(x)

x_depth = layers.Conv2D(32, 3, padding='valid',activation='elu')(x_16_pool_depth)
x2_depth = layers.Conv2D(32, 3, padding='same',activation='tanh', 
                         kernel_regularizer=reg)(x_depth)
x_depth = layers.add([x_depth, x2_depth])#
x_depth = layers.BatchNormalization()(x_depth)#

print(x_depth)
# Convolution is followed by max-pooling layer with a 2x2 window
print("48-> 24")#48->24
x_32_pool = layers.MaxPooling2D(2)(x)
print(x_32_pool)

x_32_pool_depth = layers.AveragePooling2D(2)(x_depth)
print(x_32_pool_depth)

x = layers.Dropout(0.25)(x_32_pool)
x_depth = layers.Dropout(0.2)(x_32_pool_depth)
#x_combined = layers.add([x,x_depth])

 #Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, padding = 'same', activation='elu')(x) 
#x = residual_block(x, num_filters=64)#addedclosed
x3 = layers.Conv2D(64, 3, padding = 'same', activation='elu', kernel_regularizer=reg)(x)#removed+2closed
x = layers.add([x, x3])
x = layers.BatchNormalization()(x)
print(x)

x_depth = layers.Conv2D(64, 3, padding='same',activation='elu')(x_32_pool_depth)
x2_depth = layers.Conv2D(64, 3, padding='same',activation='tanh', kernel_regularizer=reg)(x_depth)
x_depth = layers.add([x_depth, x2_depth])
x_depth = layers.BatchNormalization()(x_depth)

print(x_depth)
print("24->12") #24->12
x_64_pool = layers.MaxPooling2D(2)(x)
x_64_pool_depth = layers.AveragePooling2D(2)(x_depth)

print(x)
x_64_pool = layers.Dropout(0.25)(x_64_pool)
x_64_pool_depth = layers.Dropout(0.2)(x_64_pool_depth)

#x=layers.BatchNormalization()(x)

x = layers.Conv2D(128, 3, padding = 'same', activation='elu')(x_64_pool)
# = residual_block(x, num_filters=128) #addedclosed
x2 = layers.Conv2D(128, 3, padding = 'same', activation='elu', 
                   kernel_regularizer=reg)(x)#removed+1closed
x = layers.add([x,x2])
print("12->6") #12->6
x_128_pool = layers.MaxPooling2D(2)(x)
print(x_128_pool)

x_128_pool = layers.Dropout(0.25)(x_128_pool)

x = layers.Conv2D(192, 3, padding = 'same', activation='elu', 
                  kernel_regularizer=reg)(x_128_pool)
print(x)
x_192_pool = layers.MaxPooling2D(2)(x)
print(x_192_pool)

x_192_pool = layers.Dropout(0.25)(x_192_pool)

x = layers.Conv2D(192, 2, padding = 'valid', activation='elu', 
                  kernel_regularizer=regularizers.l2(0.01))(x_192_pool)
print(x)
#Alayers.BatchNormalization()(x) #addedclosed
x = layers.MaxPooling2D(2)(x)
print(x)

x = layers.Dropout(0.25)(x)

#x = layers.Concatenate()([x,depth_input_new])

In [ ]:
#print(depth_input)
#Define deconvolution layers
inverse = layers.Conv2DTranspose(filters = 192, kernel_size=(3,3), strides=(2, 2), padding='valid', 
                                 activation = 'elu', kernel_regularizer=reg)(x)
print(inverse)

inverse = layers.Concatenate()([inverse,x_192_pool])

inverse = layers.Conv2D(filters=192, kernel_size=(2,2), padding='same', activation='elu',
                       kernel_regularizer=reg)(inverse)
print(inverse)
inverse = layers.BatchNormalization()(inverse)

inverse = layers.Dropout(0.25)(inverse)



print(inverse)

inverse = layers.Conv2DTranspose(filters = 128, kernel_size=(2,2), strides=(2, 2), padding='valid',
                                 kernel_regularizer=reg, activation = 'elu')(inverse)
print("ok ",inverse)

inverse = layers.Concatenate()([inverse,x_128_pool])
print("concatination ",inverse)
inverse = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='elu')(inverse)
inverse = layers.BatchNormalization()(inverse)

inverse = layers.Dropout(0.25)(inverse)

inverse = layers.Conv2DTranspose(filters = 64, kernel_size=(2,2), strides=(2, 2), 
                                 padding='valid', activation = 'elu')(inverse)
print(inverse)

inverse = layers.Concatenate()([inverse,x_64_pool])

inverse = layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='elu',
                       kernel_regularizer=reg)(inverse)
inverse = layers.BatchNormalization()(inverse)

inverse = layers.Dropout(0.25)(inverse)

#inverse = layers.Concatenate()([inverse,x_64_pool_depth])

#inverse = layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='elu')(inverse)
#inverse = layers.BatchNormalization()(inverse)

#inverse = layers.Dropout(0.25)(inverse)

inverse = layers.Conv2DTranspose(filters = 32, kernel_size=(2,2), strides=(2, 2), 
                                 padding='valid', activation = 'elu')(inverse)
print(inverse)

inverse = layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='elu',
                       kernel_regularizer=reg)(inverse)
inverse = layers.Concatenate()([inverse,x_32_pool])
inverse = layers.BatchNormalization()(inverse)

inverse = layers.Dropout(0.25)(inverse)

#inverse = layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='elu')(inverse)
#x_combined = layers.Concatenate()([inverse,x_depth])
#print("Combined ", x_combined)
#x_combined = layers.add([x_,x_depth])

#x_combined = layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='elu')(x_combined)
#x_combined = layers.BatchNormalization()(x_combined)

inverse = layers.Conv2DTranspose(filters = 16, kernel_size=(4,4), strides=(2, 2), 
                                 padding='valid', activation = 'elu')(inverse)
print(inverse)

inverse = layers.Concatenate()([inverse,x_16_pool])
inverse = layers.Concatenate()([inverse,depth_input])

inverse = layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='elu')(inverse)
inverse = layers.BatchNormalization()(inverse)

inverse = layers.Dropout(0.25)(inverse)

print(inverse)

#inverse = layers.Concatenate()([inverse,x_16_pool_depth])

#inverse = layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='elu')(inverse)
#inverse = layers.BatchNormalization()(inverse)

#inverse = layers.Dropout(0.2)(inverse)


inverse2 = layers.Conv2DTranspose(filters = 1, kernel_size=(3,3), strides=(2, 2), 
                                  padding='valid',activation = 'sigmoid')(inverse)
print(inverse2)

#inverse2_depth = layers.Conv2DTranspose(filters = 1, kernel_size=(3,3), strides=(2, 2), 
#                                        padding='valid',activation = 'relu')(x_depth)
#print(inverse2_depth)

#concatenated = concatenate([inverse2, inverse2_depth])
#print(concatenated)

#concatenated =layers.Dense(10201, activation='sigmoid')

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully
# connected layer + sigmoid output layer

model = Model([img_input,depth_input], inverse2)
#model = Model(img_input, inverse2)
model.summary()


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Batch_size = 96
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True)

def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=Batch_size,seed = 1234, shuffle=True)
    genX2 = gen.flow(X1,X2, batch_size=Batch_size,seed = 1234, shuffle=True)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]
            
def get_iou(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    y_pred = np.round(y_pred)
    intersect = np.sum(np.logical_not(np.logical_xor(y_true, y_pred)))
    union = y_true.shape[0] #np.sum(y_true) + np.sum(y_pred)
    #print(intersect)
    return (intersect + 1e-9) / (union +1e-9)

def my_iou_metric(label, pred):
    return tf.py_func(get_iou, [label, pred>0.5], tf.float64)

In [ ]:
import keras.losses as losses
def dice_coeff(y_true, y_pred):
    smooth = 0.00001
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
def bce_dice_loss(y_true, y_pred):
    loss = 0.2 * losses.binary_crossentropy(y_true, y_pred) + 0.8 * dice_loss(y_true, y_pred)
    #loss = dice_loss(y_true, y_pred)
    return loss



In [ ]:
from sklearn.model_selection import train_test_split

train_x1,val_x,depth_np1, val_depth, train_y1, val_y = train_test_split(
    train_x, depth_np, train_y, test_size=0.25, random_state=1234)
epochs = 60
from  keras.optimizers import Adam

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics= ['acc'])#removed
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coeff, 'acc'])#added
from keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(monitor='dice_coeff', patience=3, mode='max')
#model.fit([train_x,depth_np],train_y, epochs=epochs,verbose=1, batch_size=96,
#callbacks = [early_stopping])#removed
#save_model_path = '/tmp/weights.hdf5'#added+2
#cp = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, 
 #                                       monitor='val_dice_loss', save_best_only=True, verbose=1)
history = model.fit([train_x,depth_np],train_y, #added
                  # steps_per_epoch=int(np.ceil(num_train_examples / float(batch_size))),
                   batch_size = Batch_size,
                   epochs=epochs
                   #validation_data=([val_x,val_depth],val_y),
                   #validation_steps=int(np.ceil(num_val_examples / float(batch_size))),
                   ,callbacks=[early_stopping]
                   )

#model.fit(train_x,train_y, epochs=epochs,verbose=1, batch_size=96, callbacks = [early_stopping])
#reduce_lr = ReduceLROnPlateau(monitor='dice_loss', mode = 'max',factor=0.5, patience=5, 
#                              min_lr=0.0001, verbose=1)

#gen_flow =

#model.fit_generator( gen_flow_for_two_inputs(train_x, depth_np, train_y) , #validation_data= ([x_val, depth_np_val],y_val),
      #                   steps_per_epoch=len(train_x) / Batch_size,                          
       #                  epochs=epochs, callbacks = [early_stopping], verbose = 1 )

model.save('model_7.h5')

In [ ]:
print("Predicing the validation set....")
val_x_pred =model.predict([val_x, val_depth])
print("Validation set prediction completed.")

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(16, 8))
columns = 3
rows = 4
%matplotlib inline
fig, axs = plt.subplots(rows, columns)
index = np.random.randint(0,1000,rows)
#print(y_pred[0,:,:,0].shape)
#index = [206, 831, 524, 676]
val_x_pred_new = np.round(val_x_pred-0.25)#-0.05)
print(index)
#plt.imshow(y_pred[100,:,:,0])
for i in range(4):
    #img = np.random.randint(10, size=(h,w))
    #fig.add_subplot(rows, columns, i)
        
    axs[i, 0].imshow(val_x[index[i],:,:,0])#, cmap=cmap)
    axs[i,1].imshow(val_y[index[i],:,:,0])#, cmap=cmap)
    axs[i,2].imshow(val_x_pred_new[index[i],:,:,0])#, cmap=cmap)
    #plt.imshow(y_pred_new[x[0],:,:,0])
#plt.show()

In [ ]:

#lmodel = load_model('model_7.h5')
df_test = df_depths[~df_depths['id'].isin(df_train['id'])]

print("Length of df_test is ", len(df_test))
df_test = df_test.reset_index()
df_test.pop('index')
print(df_test.head())

#df_test['images'] = 


In [ ]:
df_test['images'] = [np.array(Image.open("../input/test/images/{}.png".format(idx))) for 
                               idx in df_test['id']]


In [ ]:
#print(df_test.head())
print("Sample Image Shape is ", df_test['images'][10].shape)
#print("Sample Image Shape is ", df_test.loc['images'][0].shape)

In [ ]:
df_test['images'] = pd.Series(map(lambda x: np.delete(x,np.s_[1:],2), df_test['images']))
print("After optimization,  Image Shape is ", df_test['images'][0].shape)
print("No. of test images are ", len(df_test))
print("Before normalization,  pixel value is ", df_test['images'][10][2,0])
df_test['images']=df_test['images'] / 255
print("After normalization,  pixel value is ", df_test['images'][10][2,0])

print(df_test.columns)

test_x = np.array(df_test['images'])
test_x = np.concatenate(test_x,axis=0)
test_x = np.reshape(test_x,(18000,101,101,1))
print(test_x.max())
print(test_x.min())
print(train_x.max())
print(train_x.min())
print("Train Shape = ",test_x.shape)
print("Sample Pixel VAlue", test_x[10,2,0])
print(df_test.columns)



In [ ]:
df_test['depth_image'] = pd.Series(map(lambda x:np.full(shape=(50,50,1), 
                                                         fill_value=x), df_test['z']))

depth_test_np = np.array(df_test['depth_image'])
depth_test_np = np.concatenate(depth_test_np,axis=0)
depth_test_np = np.reshape(depth_test_np,(18000,50,50,1))

#print(a.shape)
#depth_np= depth_np/depth_np.max()
#print(a[2][2])
print(df_test.columns)
print(depth_test_np.shape)



In [ ]:
def post_process(x):
    num_files= x.shape[0]
    main_list = []
    print('x shape ', x.shape)
    pic_size = Height * Width
    for i in range(num_files):
        pic = x[i].T.flatten()
        s = ''
        length = 0
        start = 0
        for j in range(pic_size):
            if (j == pic_size-1):
                if((pic[j]==1) and (length==0)):
                    #s = s + ' ' + str(j) + ' 1'
                    s="{} {} 1".format(s, str(j))
                if( (pic[j]==1) and (length != 0)):
                    #s =s + ' ' + str(start) + ' ' + str(length + 1)
                    s="{} {} {}".format(s, str(start), str(length+1))

            else: #if(j != pic_size-1):
                if(pic[j]==1):
                    length += 1
                    if(length==1):
                        start = j+1
                
            if ((pic[j]==0) and (length>0)):
                #s = s + ' ' + str(start) + ' ' + str(length)
                s = "{} {} {}".format(s, str(start), str(length))
                length = 0
        main_list.append(s)
    return main_list

In [ ]:
#Make the prediction
print("Predicting now.....")
y_pred = model.predict([test_x, depth_test_np])
print("Prediction Complete")

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(8, 8))
columns = 1
rows = 4
%matplotlib inline
print(y_pred[0,:,:,0].shape)
y_pred_new = np.round(y_pred-0.25)
#plt.imshow(y_pred[100,:,:,0])
for i in range(1, 4):
    #img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    x = np.random.randint(0,4000,1)
    print(x[0])
    plt.imshow(y_pred_new[x[0],:,:,0])
plt.show()

In [ ]:
#Convert to submission format 
for i in range(5,7):
    #y_pred=np.round(y_pred-0.25).astype(int)
    print(i)
    
    new_y_pred = post_process(np.round(y_pred- (i * 0.05)).astype(int))
    output = pd.DataFrame( data = {'id': df_test['id'], 'rle_mask' : new_y_pred} )
    file_name = "output_"+ str(i) +".csv"
    print(file_name)
    output.to_csv(file_name, index=False)
#print("Y Prediction shape =",y_pred.shape)
#print(y_pred.min())
#print(y_pred.max())
#print("Y Prediction length =",len(new_y_pred))